<a href="https://colab.research.google.com/github/Thiashopy04/PROSES-MINING/blob/main/BAGIAN_B_%E2%80%94_Event_Log_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# load dataset
df = pd.read_csv('/content/1767518350948_event_log_rumah_sakit_sintetis.csv')

In [4]:
# cek missing value
df.isna().sum()

,0
case_id,0
activity,0
timestamp,0
resource,0
lifecycle,0
department,0
insurance_type,0
severity,0
cost_idr,0
variant,0


In [5]:
kolom_utama = ['case_id', 'activity', 'timestamp', 'resource']

df_clean = df.dropna(subset=kolom_utama)

In [6]:
df_clean = df_clean.drop_duplicates(
    subset=['case_id', 'activity', 'timestamp']
)

print("Jumlah data setelah cleaning:", len(df_clean))

Jumlah data setelah cleaning: 4126


In [7]:
!pip install pm4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.5 MB/s eta 0:00:00


In [8]:
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter

In [9]:
df_pm = df_clean.rename(columns={
    'case_id': 'case:concept:name',
    'activity': 'concept:name',
    'timestamp': 'time:timestamp',
    'resource': 'org:resource'
})

In [10]:
df_pm = dataframe_utils.convert_timestamp_columns_in_df(df_pm)

event_log = log_converter.apply(df_pm)

In [11]:
type(event_log)

pm4py.objects.log.obj.EventLog

In [12]:
event_per_case = df_pm.groupby('case:concept:name').size()

In [13]:
valid_case_ids = event_per_case[event_per_case >= 2].index

df_pm_valid = df_pm[df_pm['case:concept:name'].isin(valid_case_ids)]

In [14]:
event_log_final = log_converter.apply(df_pm_valid)

print("Jumlah case valid:", df_pm_valid['case:concept:name'].nunique())
print("Jumlah event final:", len(df_pm_valid))

Jumlah case valid: 600
Jumlah event final: 4126


In [15]:
from pm4py.statistics.traces.generic.log import case_statistics

stats = case_statistics.get_all_case_durations(event_log_final)

print("Durasi case (detik):")
print("Min   :", min(stats))
print("Max   :", max(stats))
print("Rata2 :", sum(stats)/len(stats))

Durasi case (detik):
Min   : 180.0
Max   : 321541.0
Rata2 : 39362.056666666664


✍️ Narasi Singkat untuk Laporan (BAGIAN B)

Pada tahap event log preparation, dilakukan pembersihan data dengan menghapus missing value pada kolom utama dan menghilangkan event duplikat.
Dataset kemudian dikonversi ke format event log PM4Py dengan standarisasi nama kolom.
Validasi dilakukan untuk memastikan setiap case memiliki minimal dua aktivitas sehingga layak untuk dianalisis lebih lanjut dalam process mining. **bold text**